In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
# Item Rando:
# 26 'randomizeable' items
# 100 'randomizeable' drop locations
# idea 1: location : item_id
# ~ 500 bits (locations * log(items))
# idea 2: item : location_id
# ~ 700 bits (locations * log(locations))
# idea 3: location : common_item_id
# major_item : location_id
# 300 bits + 126 bits

In [3]:
# Door Rando:
# ~550 doors
# idea 1: door: door_id
# 550 * log(550) = 5500 bits
# idea 2: left_door : right_door_id
# top_door : bottom_door_id
# 225 * log(225) = 2040
# idea 3:
# X, Y
# overworld is ~ 128 x 128
# 14 bits per room * 255 rooms = 3570

In [4]:
import sys
sys.path.append("..")

In [5]:
from encoding.parse_rooms import parse_rooms, parse_exits, dictify_rooms

In [6]:
from functools import cache

In [7]:
rooms = parse_rooms("../encoding/dsl/rooms_for_alloy.txt")
exits = parse_exits("../encoding/dsl/exits_for_alloy.txt")
design = dictify_rooms(rooms, exits)

In [8]:
from rom_tools.rom_manager import RomManager
from abstraction_validation.sm_paths import *

In [9]:
from bdds.bdd_core import *
from bdds.node_bdds import *

In [10]:
from functools import reduce

In [11]:
all_nodes = []
for r, room in rooms.items():
    for node in room.graph.name_node.keys():
        all_nodes.append(node)
node_ids = {n:i for i,n in enumerate(all_nodes)}

In [12]:
# Issue with design translation via chopping off the last _
node_ids["Spore_Spawn_Spawn"] = node_ids["Spore_Spawn_Spore_Spawn"]
node_ids["Golden_Torizo_Torizo"] = node_ids["Golden_Torizo_Golden_Torizo"]
node_ids["Mother_Brain_Brain"] = node_ids["Mother_Brain_Mother_Brain"]

In [13]:
def loc_id(room_name, node_name, when="prev"):
    node_id = node_ids[f"{room_name}_{node_name}"]
    return f"node_id_{when} = {node_id}"

In [14]:
rando_items = ["B", "MB", "PB", "SPB", "S", "M", "G", "SA", "V", "GS", "SB", "HJ", "CB", "WB", "E", "PLB", "Spazer", "RT", "XR", "IB", "SJ"]
# TODO: which items are unique?

In [15]:
len(rando_items)

21

In [16]:
context_id = mk_context_id(node_ids)

In [17]:
#TODO: make this more efficient
@cache
def item_transitions(item_gained=None):
    if item_gained is None:
        return "(items_unchanged)"
    clauses = []
    for i in design["Items"] | design["Bosses"]:
        if i == item_gained:
            clause = f"{i}_prev < {i}_next"
        else:
            clause = f"{i}_prev = {i}_next"
        clauses.append(clause)
    return " & ".join(clauses)

def itemset_to_str(itemset):
    if len(itemset) == 0:
        return "TRUE"
    else:
        return " & ".join([f"{item}_prev = 1" for item in itemset])

def required_itemsets(itemsets):
    return "(" + " | ".join([itemset_to_str(itemset) for itemset in itemsets]) + ")"

def rando_transitions(room_name, node_name):
    t = context_id.false
    node_name = room_name + "_" + node_name
    drop_name = f"drop_{node_name}"
    for i,item in enumerate(rando_items):
        t |= context_id.add_expr(f"{drop_name} = {i}") & context_id.add_expr(item_transitions(item))
        #t |= context.add_expr(f"{drop_name} = {i} &" + item_transitions(item))
    return t

In [18]:
drop_nodes = []
for room_name, room in design["Rooms"].items():
    for node_name, d in room["Drops"].items():
        if d in rando_items:
            drop_nodes.append(room_name + "_" + node_name)

In [19]:
assert len(drop_nodes) == 100

In [20]:
major_item_nodes = []
for room_name, room in design["Rooms"].items():
    for node_name, d in room["Drops"].items():
        if d in rando_items:
            drop_nodes.append(room_name + "_" + node_name)

In [21]:
#limit design freedom by setting a limited number of locations
#drop_nodes = ["Bomb_Torizo_Bombs", "Big_Pink_M2"]
n_randomizeable_drops = 4
drop_nodes = drop_nodes[:n_randomizeable_drops]
#drop_nodes = []

In [22]:
design_vars = {
    f"drop_{node}": (0,len(rando_items)-1) for node in drop_nodes
}

In [23]:
if len(design_vars) > 0:
    context_id.declare(**design_vars)

In [24]:
def combine_bdd_group(bdds):
    return reduce(lambda x,y: x|y, bdds, context_id.false)

In [25]:
# Build individual BDDs
room_bdds = []
door_bdds = []
for room_name, room in tqdm(design['Rooms'].items()):
    links = []
    for node_name in room['Nodes']:
        s = loc_id(room_name, node_name) + " & " + loc_id(room_name, node_name, when="next") + " & " + item_transitions()
        links.append(context_id.add_expr(s, with_ops=True))
        if node_name in room['Drops']:
            if room_name + "_" + node_name in drop_nodes:
                s = context_id.add_expr(loc_id(room_name, node_name) + " & " + loc_id(room_name, node_name, when="next")) & rando_transitions(room_name, node_name)
            else:
                s = context_id.add_expr(loc_id(room_name, node_name) + " & " + loc_id(room_name, node_name, when="next") + " & " + item_transitions(room['Drops'][node_name]), with_ops=True)
            links.append(s)
    for node_name, door in room['Doors'].items():
        d = loc_id(room_name, node_name) + " & " + loc_id(door['Room'], door['Node'], when="next") + " & " + item_transitions()
        door_bdds.append(context_id.add_expr(d, with_ops=True))
    for node_name, edges in room['Edges'].items():
        for edge in edges:
            other_node_name = edge['Terminal']
            s = loc_id(room_name, node_name) + " & " + loc_id(room_name, other_node_name, when="next") + " & " + required_itemsets(edge['Requirements']) + " & " + item_transitions()
            links.append(context_id.add_expr(s, with_ops=True))
    room_bdd = combine_bdd_group(links)
    #print(room_bdd.count())
    room_bdds.append(room_bdd)

100%|█████████████████████████████████████████████████████████████████████████████████| 255/255 [00:13<00:00, 18.36it/s]


In [26]:
doors_bdd = combine_bdd_group(door_bdds)
rooms_bdd = combine_bdd_group(room_bdds)
all_bdd = doors_bdd | rooms_bdd

In [27]:
all_bdd.dag_size

20301

In [28]:
all_bdd.count()
context_id.exist(prevs, all_bdd).count()

349.0

In [29]:
len(all_bdd.support)

112

In [30]:
def get_reachable_id(trans_norule, context, start_bdd):
    n = 0
    covered_p = start_bdd
    covered_last_p = context.false
    while covered_p != covered_last_p:
        covered_last_p = covered_p
        fringe_n = context.exist(prevs, covered_p & trans_norule)
        covered_p |= context.let(next_to_prev, fringe_n)
        print(n, covered_p.dag_size)
        n+=1
    return covered_p

In [31]:
def mk_closure(trans, context, max_steps=float("inf")):
    n = 0
    closure = trans
    closure_last = context.false
    while closure != closure_last and n < max_steps:
        closure_last = closure
        closure_prev_temp = context.let(next_to_temp, closure_last)
        closure_temp_next = context.let(prev_to_temp, closure_last)
        closure |= context.exist(temps, closure_prev_temp & closure_temp_next)
        print(n, closure.dag_size)
        n+=1
    closure_square = closure
    return closure_square

In [32]:
from data_types import item_set

In [33]:
def mk_itemset_expr(itemset, when="prev"):
    clauses = []
    for i in item_mapping:
        if i in itemset:
            clause = f"{i}_{when} = 1"
        else:
            clause = f"{i}_{when} = 0"
        clauses.append(clause)
    return " & ".join(clauses)

In [34]:
start_bdd = context_id.add_expr("node_id_prev = 0 & " + mk_itemset_expr(item_set.ItemSet()))

In [35]:
all_items_bad = context_id.true
for drop in design_vars.keys():
    all_items_bad &= context_id.add_expr(f"{drop} < 4")

In [36]:
context_id.bdd.configure()

{'reordering': False,
 'garbage_collection': True,
 'max_memory': 18446744073709551615,
 'loose_up_to': 6710886,
 'max_cache_soft': 6716416,
 'max_cache_hard': 4294967295,
 'min_hit': 30,
 'max_growth': 1.2,
 'max_swaps': 2000000,
 'max_vars': 1000}

In [37]:
log2(18446744073709551615)

64.0

In [38]:
%%time
all_bdd_fast = mk_closure(all_bdd, context_id, 3)

0 40125
1 144756
2 641828
CPU times: user 5.3 s, sys: 56.6 ms, total: 5.36 s
Wall time: 5.34 s


In [ ]:
%%time
reachable = get_reachable_id(all_bdd_fast, context_id, start_bdd)

0 60
1 109
2 149
3 146
4 120
5 155
6 168
7 232
8 358
9 349
10 444
11 480
12 622
13 748
14 868
15 756
16 801
17 803
18 909
19 1207
20 1250
21 1483
22 1720
23 2117
24 2616
25 3030
26 3843
27 5145
28 8407
29 11779
30 27695
31 42693
32 85790
33 153107
34 243871
35 348040
36 485502
37 607904
38 682856
39 759656
40 851768
41 1001711
42 1200435
43 1466865
44 1798421
45 2104724
46 2400788
47 2626458
48 2786398
49 2944182
50 3042159
51 3157035
52 3253476
53 3434676
54 3619014
55 3709492
56 3682242
57 3481154


In [ ]:
#TODO: Iterative Squaring

In [ ]:
end_reachable = context_id.add_expr("node_id_prev = 1")

In [ ]:
g_wo_s = context_id.add_expr("G_prev = 1 & S_prev = 0")

In [ ]:
(end_reachable & reachable).count()

In [ ]:
#TODO: ensure no invalid node ids

In [ ]:
get_g_wo_s = context_id.exist(prevs, reachable & g_wo_s)
get_end = context_id.exist(prevs, reachable & end_reachable)

In [ ]:
(get_g_wo_s & get_end).dag_size

In [ ]:
for sol in context_id.pick_iter(get_g_wo_s & get_end):
    td = {}
    for k,v in sol.items():
        if v < len(rando_items):
            td[k] = rando_items[v]
        else:
            td[k] = v
    print(td)

In [ ]:
# Gaussian distribution over shortest path times
# Action n-grams
# Major/Minor rando
# nx.draw layout

In [ ]:
#TODO: Region-based or item-collection based cuts
# Saturate the space ahead-of-time